In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import zipfile

datapath = '/content/drive/MyDrive/CV project personal/Copy of train.zip'

# Unzip training data
zip_ref = zipfile.ZipFile(datapath, 'r')
zip_ref.extractall('/content/data')
zip_ref.close()

# Unzip testing data
#zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Colab/test.zip', 'r')
#zip_ref.extractall('/content/data')
#zip_ref.close()

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as f
import matplotlib.pyplot as plt
import numpy as np
import json
from PIL import Image
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
import torch.optim as optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [4]:
cuda = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# initialize model to random weights
classifier = efficientnet_v2_s().to(cuda)
loss_func = f.cross_entropy
opt = optim.Adam(classifier.parameters(), lr=1e-3)

transform = EfficientNet_V2_S_Weights.DEFAULT.transforms()

In [5]:
with open('/content/drive/MyDrive/Computer Vision Project/train_annotation.json') as f:
  train_annotations = json.load(f)

In [6]:
class_dict = {}
train_labels = []

for data in train_annotations['data']:
  species = data['species']
  if species not in class_dict.keys():
    class_dict[species] = 0
  class_dict[species] += 1
  train_labels.append(species)

label_to_id = {}
for i, species in enumerate(class_dict):
  label_to_id[species] = i

classes = list(class_dict.keys())

train_label_ids = [label_to_id[label] for label in train_labels]
train_label_ids = torch.tensor(train_label_ids)

In [7]:
def append_tensors(tList):
  outList = []
  for tensor in tList:
    outList.append(tensor.detach().numpy())
  outList = np.array(outList)
  return torch.tensor(outList)

# train_im = []
# for data in train_annotations['data']:
#   train_im.append(transform(Image.open('/content/data/train/' + data['file'])))
# train_im = append_tensors(train_im)

train_im_name_ind = [i for i in range(len(train_annotations['data']))]
train_im_name_ind = torch.tensor(train_im_name_ind)

train_dataset = TensorDataset(train_im_name_ind, train_label_ids)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [8]:
epochs = 5
classifier.train()
for epoch in range(epochs):
  running_loss = 0.0
  for i, (inputs, labels) in enumerate(train_dataloader):
    # load batch
    batch_im = []
    for file_ind in inputs:
      batch_im.append(transform(Image.open('/content/data/train/' + train_annotations['data'][file_ind]['file'])))
    batch_im = append_tensors(batch_im)

    labels = labels.to(cuda)
    batch_im = batch_im.to(cuda)
    opt.zero_grad()
    pred = classifier(batch_im)
    loss = loss_func(pred, labels)
    loss.backward()
    opt.step()

    running_loss += loss.item()
    if i % 50 == 49:
      print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
      running_loss = 0.0

print('Finished Training')

[1,    50] loss: 0.076
[1,   100] loss: 0.050
[1,   150] loss: 0.053
[1,   200] loss: 0.050
[1,   250] loss: 0.048
[1,   300] loss: 0.046
[1,   350] loss: 0.043
[1,   400] loss: 0.045
[1,   450] loss: 0.044
[1,   500] loss: 0.046
[1,   550] loss: 0.042
[1,   600] loss: 0.043
[1,   650] loss: 0.043
[1,   700] loss: 0.042
[1,   750] loss: 0.043
[1,   800] loss: 0.046
[1,   850] loss: 0.044
[1,   900] loss: 0.044
[1,   950] loss: 0.043
[1,  1000] loss: 0.041
[1,  1050] loss: 0.040
[1,  1100] loss: 0.040
[1,  1150] loss: 0.041
[1,  1200] loss: 0.041
[1,  1250] loss: 0.039
[1,  1300] loss: 0.039
[1,  1350] loss: 0.040
[1,  1400] loss: 0.040
[1,  1450] loss: 0.039
[1,  1500] loss: 0.040
[1,  1550] loss: 0.039
[1,  1600] loss: 0.042
[1,  1650] loss: 0.041
[1,  1700] loss: 0.040
[1,  1750] loss: 0.037
[1,  1800] loss: 0.039
[1,  1850] loss: 0.040
[1,  1900] loss: 0.039
[1,  1950] loss: 0.036
[1,  2000] loss: 0.037
[1,  2050] loss: 0.039
[1,  2100] loss: 0.038
[1,  2150] loss: 0.039
[1,  2200] 

KeyboardInterrupt: ignored

In [ ]:
torch.save(classifier.state_dict(), '/content/drive/MyDrive/CV project personal/classifier_model.pth')